In [ ]:
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import numpy as np

In [ ]:
def load(m):
    with open(f'/home/nahappy15/accepTAble/sample_stat_gaussian_noise_{m}.pkl', 'rb') as f:
        c_list = pickle.load(f)
    return m, c_list

In [ ]:
method_list = ['noadapt', 'eata', 'eata_aug', 'sar', 'sar_aug','deyo', 'deyo_aug',]
data = {}
for m in method_list:
    k, v = load(m)
    data[k] = v

In [ ]:
def count_class(data_list):
    ent_num = np.zeros((1000,))
    correct_num = np.zeros((1000,))
    data_num = np.zeros((1000,), dtype=np.float32)
    correct_rate = np.zeros((1000,), dtype=np.float32)

    for _, ef, lbl, cor in data_list:
        for i in ef:
            ent_num[i] += 1
        for l, c in zip(lbl, cor):
            if c:
                correct_num[l] += 1
            data_num[l] += 1
    
    data_num[data_num==0] = 1
    correct_rate = correct_num / data_num
    return ent_num, correct_num, correct_rate, data_num

In [ ]:
exp = 'deyo'
e0, c0, r0, n0 = count_class(data['noadapt'])
en, cn, cr, nr = count_class(data[exp])
ena, cna, cra, nra = count_class(data[f'{exp}_aug'])

In [ ]:
df = pd.DataFrame([e0, c0, en, ena, cn, cna, cr, cra, nr, nra], 
                  index=['e0', 'c0', 'ent_num', 'ent_num_aug', 'correct_num', 'correct_num_aug', 'correct_rate', 'correct_rate_aug', 'num', 'num_aug']).T
df['ie0'] = df['e0'] // 10
df['ie'] = df['ent_num'] // 10
df['iea'] = df['ent_num_aug'] // 10
df

In [ ]:
dff = df.drop(['correct_rate', 'correct_rate_aug'], axis=1)

dff = pd.DataFrame([
    dff.groupby('ie0').mean()['c0'] / 50.,
    dff.groupby('ie').mean()['correct_num'] / 50.,
    dff.groupby('iea').mean()['correct_num_aug'] / 50.
]).T
dff

In [ ]:
dff.plot(kind='bar')

In [ ]:
df.plot(x='ie', y=['e0', 'ent_num', 'ent_num_aug'], kind='hist')
# df.plot(x='ie', y=['correct_rate', 'correct_rate_aug'], kind='hist')

In [ ]:
df.describe()

In [ ]:
df.hist(['e0', 'ent_num', 'ent_num_aug'], bins=10)

In [ ]:
df.hist('e0', bins=16) #65% of data has only <5 samples -- y: # class, x: # samples

In [ ]:
df['e0'].to_csv('e0.csv', index=False)

In [ ]:
df['ec0'] = df['e0'] // 5

In [ ]:
df[['e0', 'ec0', 'c0']].groupby('ec0').mean() * 2

In [ ]:
dff[['cr0', 'correct_rate', 'correct_rate_aug']].to_csv('r0.csv')

In [ ]:
df[['e0', 'c0']].plot(x='e0', y='c0', kind='scatter', s=1)

In [ ]:
df[['e0', 'c0']].corr()